In [ ]:
!pip install opendatasets
!pip install geopy
!pip install folium

#### load dataset with kaggle in google colab

In [ ]:
import opendatasets
opendatasets.download('https://www.kaggle.com/datasets/yelp-dataset/yelp-dataset')

#### mount google drive in google colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import json
import pandas as pd

#### load dataset

In [ ]:
data_file = open("/content/drive/MyDrive/yelp-dataset/yelp-dataset/yelp_academic_dataset_business.json")
data = []
for line in data_file:
  data.append(json.loads(line))
business_df = pd.DataFrame(data)
business_df

In [ ]:
city = business_df[business_df["city"]=="Santa Barbara"]
city

In [ ]:
data_file = open("/content/drive/MyDrive/yelp-dataset/yelp-dataset/yelp_academic_dataset_review.json")
data = []
with open('/content/yelp_review.json', 'w') as file:

  for line in data_file:
    row = json.loads(line)
    if (row["business_id"] in list(city["business_id"])):
      file.write(line)

In [ ]:
# !cp /content/yelp_review.json /content/drive/MyDrive/yelp-dataset/yelp-dataset

In [ ]:
data_file = open("/content/drive/MyDrive/yelp-dataset/yelp-dataset/yelp_review.json")
data = []


for line in data_file:
  data.append(json.loads(line))

review_df = pd.DataFrame(data)
review_df

#### insert price for business

In [ ]:
#اضافه کردن دستی قیمت

city["price"] = 0
import random
indexes = city.index
for index in indexes:
  city.loc[index, "price"] = random.randint(1, 100)

city

#### calculate distance and time

In [ ]:
from geopy.distance import geodesic

In [ ]:
source = (41.49008, -71.312796)
destination = (32.223236, -110.880452)
distance = geodesic(source, destination).km
print(f"distance: {distance}")
#km
avrage_speed = 90 #km
#hours
time =  distance / avrage_speed  

print(f"time: {time}")

In [ ]:
def fitnees_time(solution):

  time = 0
  avrage_speed = 80 #km
  total_distance = 0

  for i in range(len(solution)-1):
    current = city[city['business_id'] == solution[i]]
    next = city[city['business_id'] == solution[i+1]]


    source = np.zeros(2, dtype=float)
    source[0] = current["latitude"]
    source[1] = current["longitude"]

    destination = np.zeros(2, dtype=float)
    destination[0] = next["latitude"]
    destination[1] = next["longitude"]

    total_distance += geodesic(source, destination).km

  print(f"total time: {int(avrage_speed / total_distance )}h")
  print(f"total distance: {int(total_distance)}km")


In [ ]:
def fitnees_price(solution):

  total_price = 0

  for i in range(len(solution)):

    current = city[city['business_id'] == solution[i]]

    total_price += int(current["price"])

  print(f"total price: {total_price}")

In [ ]:
import numpy as np

In [ ]:
population = np.empty((100,10), dtype=object)
pareto_ranking = np.empty((100, 2), dtype=object)

In [ ]:
for i in range(100):
  index = np.random.choice(city.index, replace=False, size=10)
  selected_row = np.array(city.loc[index, "business_id"])
  solution = np.empty(10, dtype=object)

  for j in range(len(selected_row)):
      solution[j] = selected_row[j]


  population[i] = solution


In [ ]:
for i in range(len(population)):
  s = population[i]

  pareto_ranking[i, 0] = fitnees_price(s)
  pareto_ranking[i, 1] = fitnees_time(s)


pareto_ranking_df = pd.DataFrame(data=pareto_ranking, columns=['Price', 'Time'])
pareto_ranking_df["rank"] = 0

In [ ]:
pareto_ranking_array = np.array(pareto_ranking_df.loc[:, ['Time', 'Price']])

In [ ]:
for i in range(len(pareto_ranking_array)):
    for j in range(len(pareto_ranking_array)):
        if all(pareto_ranking_array[j] <= pareto_ranking_array[i]) and any(pareto_ranking_array[j] < pareto_ranking_array[i]):
           pareto_ranking_df.loc[j, "rank"] = i

pareto_ranking_df.sort_values("rank")

In [ ]:
import folium
from geopy.geocoders import ArcGIS
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="location_details")

In [ ]:
map=folium.Map(location=["34.426679", "-119.711197"],zoom_start=15)

for i in range(len(city)):
   if i == 5:
      break
   latitude = city.iloc[i]["latitude"]
   longitude = city.iloc[i]["longitude"]
   map.add_child(folium.Marker(location=[latitude, longitude], popup=i, ,icon=folium.Icon(color='blue')))
map.add_child(folium.Marker(location=["34.426679", "-119.711197"],icon=folium.Icon(color='blue')))